In [2]:
import pandas as pd
df = pd.read_csv('../Data/insurance_advanced_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
X = df[['age', 'affordibility']]
y = df['bought_insurance']

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[0.09090909, 1.        ],
       [0.15909091, 0.        ],
       [0.65909091, 1.        ],
       [0.77272727, 0.        ],
       [0.63636364, 1.        ],
       [0.86363636, 1.        ],
       [0.84090909, 0.        ],
       [0.95454545, 0.        ],
       [1.        , 1.        ],
       [0.97727273, 1.        ],
       [0.        , 1.        ],
       [0.22727273, 1.        ],
       [0.20454545, 0.        ],
       [0.25      , 0.        ],
       [0.70454545, 1.        ],
       [0.84090909, 1.        ],
       [0.15909091, 0.        ],
       [0.90909091, 1.        ],
       [0.02272727, 0.        ],
       [0.        , 1.        ],
       [0.06818182, 1.        ],
       [0.18181818, 0.        ],
       [0.5       , 1.        ],
       [0.61363636, 1.        ],
       [0.72727273, 1.        ],
       [0.81818182, 1.        ],
       [0.11363636, 1.        ],
       [0.63636364, 1.        ]])

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [42]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Input(shape=(2,)),
    keras.layers.Dense(1, activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=5000)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step - accuracy: 0.4545 - loss: 0.7324
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4545 - loss: 0.7319
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4545 - loss: 0.7314
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4545 - loss: 0.7310
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4545 - loss: 0.7305
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4545 - loss: 0.7300
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4545 - loss: 0.7295
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4545 - loss: 0.7290
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4545 - loss: 0.7285
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4545 - loss: 0.7280
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4545 - loss: 0.7276
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy

In [44]:
model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.2807


[0.28067871928215027, 1.0]

In [45]:
coef, intercept = model.get_weights()
coef, intercept

(array([[3.7983537],
        [1.4821229]], dtype=float32),
 array([-3.0378733], dtype=float32))

In [26]:
import numpy as np

def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [38]:
def gradient_descent(age, affordability, y_true, epochs):
   w1 = w2 = 1
   bias = 0
   rate = 0.5
   n = len(age)

   for i in range(epochs):
      weighted_sum = w1 * age + w2 * affordability + bias
      y_predicted = sigmoid_numpy(weighted_sum)
      loss = log_loss(y_true, y_predicted)

      w1d = (1 / n) * np.dot(np.transpose(age), (y_predicted - y_true)) 
      w2d = (1 / n) * np.dot(np.transpose(affordability), (y_predicted - y_true)) 
      bias_d = np.mean(y_predicted - y_true)

      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d
      bias = bias - rate * bias_d

      print (f'Epoch:{i + 1}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

   return w1, w2, bias

In [46]:
X_train_age = np.array([i[0] for i in X_train])
X_train_affordibility = np.array([i[1] for i in X_train])
gradient_descent(X_train_age, X_train_affordibility, y_train, 1000)

Epoch:1, w1:0.973109197719838, w2:0.9257843080104015, bias:-0.14385467928615314, loss:0.7324218477191861
Epoch:2, w1:0.9541004102270774, w2:0.8638804205244837, bias:-0.26955735258309965, loss:0.6822402893472949
Epoch:3, w1:0.942604823169715, w2:0.8137770514303034, bias:-0.37834030058483276, loss:0.6452105318649003
Epoch:4, w1:0.9379368785720064, w2:0.774434190993009, bias:-0.47197745276669334, loss:0.618501234536347
Epoch:5, w1:0.9392294571175897, w2:0.7445129790495383, bias:-0.5524948268960452, loss:0.5994421379868938
Epoch:6, w1:0.9455599171356637, w2:0.7225842957433058, bias:-0.6219289130046539, loss:0.5858021327861409
Epoch:7, w1:0.9560410067170867, w2:0.7072755652195057, bias:-0.6821684021072992, loss:0.5758732201613277
Epoch:8, w1:0.9698730159044311, w2:0.6973528534153544, bias:-0.7348744096597133, loss:0.5684275119030793
Epoch:9, w1:0.9863654802236865, w2:0.6917537431170311, bias:-0.7814569668350639, loss:0.5626206618170358
Epoch:10, w1:1.0049393560382973, w2:0.6895894640716224,

(4.6313468853330955, 2.095139774013462, -4.053149060568569)

In [47]:
coef, intercept

(array([[3.7983537],
        [1.4821229]], dtype=float32),
 array([-3.0378733], dtype=float32))